In [1]:
!pip install editdistance
!pip install torch pytorch-lightning
!pip install transformers tokenizers sentencepiece
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 10.1 MB/s eta 0:00:00


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
!pip install --upgrade wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.15.9
    Uninstalling wandb-0.15.9:
      Successfully uninstalled wandb-0.15.9


In [4]:
import time
from torch.utils.data import Dataset
import os
import os
import logging
import time
import pickle
from tqdm import tqdm
import logging
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from transformers import AdamW, MT5ForConditionalGeneration, AutoTokenizer
from transformers import get_linear_schedule_with_warmup

import re
import editdistance
import warnings
warnings.filterwarnings('ignore')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import random

def preprocessing_data(path, is_valid=False):
  list_compare_quintuple = []
  for folder in os.listdir(path):
    with open(path + "/" + folder, "r", encoding="utf-8") as file:
        all_data = file.read()
        list_compare = all_data.split("\n\n")
        for i in range(len(list_compare)):
          list_compare_quintuple.append(list_compare[i].split('\n'))

  new_list_compare_quintuple = []
  for i in range(len(list_compare_quintuple)):
      if len(list_compare_quintuple[i]) == 3:
          new_list_compare_quintuple.append(list_compare_quintuple[i])
  list_compare_quintuple = new_list_compare_quintuple

  for i in range(len(list_compare_quintuple)):
      for j in range(len(list_compare_quintuple[i])):
          list_compare_quintuple[i][j] = list_compare_quintuple[i][j].lower()
  return list_compare_quintuple

In [6]:
def convert_list_to_string(list_):
    ###
    # ['16&&nâng', '17&&cấp', '18&&nhẹ']
    # => nâng cấp nhẹ
    ###
    string_ = ''
#     print(list_)
    for i in range(len(list_)):
        string_ += list_[i].split('&&')[1] + ' '
    if string_ == '':
        return 'unknow'
    return string_[:-1]

In [7]:
import json

# senttag2word = {
#     'com+': 'comparable and positive',
#     'eql': 'equal',
#     'com-': 'comparable and negative',
#     'sup+': 'superlative and positive',
#     'dif': 'different',
#     'com': 'comparable',
#     'sup-': 'superlative and negative',
#     'sup': 'superlative',
# }
def get_data_from_quin(list_compare_quintuple):


    sents = []
    targets = []
    for i in range(len(list_compare_quintuple)):
#         print(list_compare_quintuple[i])
        sents.append(list_compare_quintuple[i][0])
        target = ""
        for j in range(1, len(list_compare_quintuple[i])):
#             print("hi",list_compare_quintuple[i][j] )
#             print(list_compare_quintuple[i][j])
            json_data = json.loads(list_compare_quintuple[i][j])
#             targets.append(list_compare_quintuple[i][j])
            subject = convert_list_to_string(json_data['subject'])
            object = convert_list_to_string(json_data['object'])
            aspect = convert_list_to_string(json_data['aspect'])
            predicate = convert_list_to_string(json_data['predicate'])
            label = (json_data['label'])
            target +=  subject + " > " + object + " > " + aspect + " > " + predicate + ";" 
        targets.append(target.strip())
    return sents, targets


In [8]:
def read_data_from_path(path, is_valid):
    list_compare_quintuple = preprocessing_data(path, is_valid)
    sents, targets = get_data_from_quin(list_compare_quintuple)
    return sents, targets


In [9]:
sents, targets = read_data_from_path("/kaggle/input/dataset1/train", is_valid=False)


In [10]:
class ABSADataset(Dataset):
    def __init__(self, tokenizer, data_path, data_type, max_len=100, is_valid=False):
        # 'data/aste/rest16/train.txt'
        self.data_path = f'{data_path}/{data_type}'
        self.data_type = data_type
        self.max_len = max_len
        self.tokenizer = tokenizer

        self.inputs = []
        self.targets = []
        self.is_valid=is_valid
        self._build_examples()


    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()      # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask,
                "target_ids": target_ids, "target_mask": target_mask}

    def _build_examples(self):

        inputs, targets = read_data_from_path(self.data_path, self.is_valid)

        for i in range(len(inputs)):

#             input = ' '.join(inputs[i])
            input = inputs[i]
            target = targets[i]


            tokenized_input = self.tokenizer.batch_encode_plus(
              [input], max_length=self.max_len, pad_to_max_length=True, truncation=True,
              return_tensors="pt",
            )
            tokenized_target = self.tokenizer.batch_encode_plus(
              [target], max_length=self.max_len, pad_to_max_length=True, truncation=True,
              return_tensors="pt"
            )

            self.inputs.append(tokenized_input)
            self.targets.append(tokenized_target)

In [11]:
class init_args:
    def __init__(self):
        self.data_path='/kaggle/input/dataset1/'
#         self.model_name_or_path='google/mt5-base'
        self.model_name_or_path='google/mt5-base'
        self.do_train=True
        self.do_direct_eval=True
        self.n_gpu='0'
        self.gradient_accumulation_steps=16
        self.learning_rate=2e-4
        self.num_train_epochs=15
        self.max_seq_length=128
        self.seed=42
        self.weight_decay=0.001
        self.adam_epsilon=1e-8
        self.warmup_steps=0
        self.num_workers=4

        self.batch_size=1

        if not os.path.exists('./outputs'):
            os.mkdir('./outputs')
        output_dir = f"./outputs"
        self.output_dir = output_dir

args = init_args()

In [12]:
def compute_f1_scores(predict_tuple, gold_tuple):
    """
    Function to compute F1 scores with pred and gold pairs/triplets
    The input needs to be already processed
    """
    # number of true postive, gold standard, predicted aspect terms
    n_tp, n_gold, n_pred = 0, 0, 0

    for i in range(len(predict_tuple)):
        n_gold += len(gold_tuple[i])
        n_pred += len(predict_tuple[i])

        for t in predict_tuple[i]:
            if t in gold_tuple[i]:
                n_tp += 1

    precision = float(n_tp) / float(n_pred) if n_pred != 0 else 0
    recall = float(n_tp) / float(n_gold) if n_gold != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision != 0 or recall != 0 else 0
    scores = {'precision': precision, 'recall': recall, 'f1': f1}

    return scores

In [13]:
def recover_terms_with_editdistance(original_term, sent):
    words = original_term.split(' ')
    new_words = []
    for word in words:
        edit_dis = []
        for token in sent:
            edit_dis.append(editdistance.eval(word, token))
        smallest_idx = edit_dis.index(min(edit_dis))
        new_words.append(sent[smallest_idx])
    new_term = ' '.join(new_words)
    return new_term

def extract_spans_extraction(seq):
    extractions = []
#     print(seq)
    # if task == 'uabsa' and seq.lower() == 'none':
    #     return []
    # else:
    #     if task in ['uabsa', 'aope']:
            # all_pt = seq.split('; ')
            # for pt in all_pt:
            #     pt = pt[1:-1]
            #     try:
            #         a, b = pt.split(', ')
            #     except ValueError:
            #         a, b = '', ''
            #     extractions.append((a, b))
        # elif task in ['tasd', 'aste']:

    all_pt = seq.split(' ; ')
    for pt in all_pt:
        pt = pt[0:-0]
        try:
            a, b, c, d = pt.split(' > ')
            extractions.append((a, b, c, d))
        except ValueError:
            a, b, c, d, e = '', '', '', '', ''

    return extractions

def fix_preds_(all_pairs, sents):
    return all_pairs
    all_new_pairs = []


    for i, pairs in enumerate(all_pairs):
        new_pairs = []
        if pairs == []:
            all_new_pairs.append(pairs)
        else:
            print("*" * 100)
            print(pairs)
            print("*" * 100)
            for pair in pairs:
                #two formats have different orders
                p0, p1, p2, p3, p4 = pair
                # for annotation-type
                if p1 in sentiment_word_list:
                    at, ott, ac = p0, p2, p1
                    io_format = 'annotation'
                # for extraction type
                elif p2 in sentiment_word_list:
                    at, ott, ac = p0, p1, p2
                    io_format = 'extraction'

                #print(pair)
                # AT not in the original sentence
                if at not in  ' '.join(sents[i]):
                    # print('Issue')
                    new_at = recover_terms_with_editdistance(at, sents[i])
                else:
                    new_at = at

                if ac not in sentiment_word_list:
                    new_sentiment = recover_terms_with_editdistance(ac, sentiment_word_list)
                else:
                    new_sentiment = ac

                # OT not in the original sentence
                ots = ott.split(', ')
                new_ot_list = []
                for ot in ots:
                    if ot not in ' '.join(sents[i]):
                        # print('Issue')
                        new_ot_list.append(recover_terms_with_editdistance(ot, sents[i]))
                    else:
                        new_ot_list.append(ot)
                new_ot = ', '.join(new_ot_list)
                if io_format == 'extraction':
                    new_pairs.append((new_at, new_ot, new_sentiment))
                else:
                    new_pairs.append((new_at, new_sentiment, new_ot))
                # print(pair, '>>>>>', word_and_sentiment)
                # print(all_target_pairs[i])
            all_new_pairs.append(new_pairs)

    return all_new_pairs


def fix_pred_with_editdistance(all_predictions, sents):
    fixed_preds = fix_preds_(all_predictions, sents)
    return fixed_preds

In [14]:
def compute_scores(pred_seqs, gold_seqs, sents):
    """
    compute metrics for multiple tasks
    """
    assert len(pred_seqs) == len(gold_seqs)
    num_samples = len(gold_seqs)

    all_labels, all_predictions = [], []

    for i in range(num_samples):

        gold_list = extract_spans_extraction(gold_seqs[i])
        pred_list = extract_spans_extraction(pred_seqs[i])

        all_labels.append(gold_list)
        all_predictions.append(pred_list)

    print("\nResults of raw output")
    raw_scores = compute_f1_scores(all_predictions, all_labels)
    print(raw_scores)

    # fix the issues due to generation
    all_predictions_fixed = fix_pred_with_editdistance(all_predictions, sents)
    print("\nResults of fixed output")
    fixed_scores = compute_f1_scores(all_predictions_fixed, all_labels)
    print(fixed_scores)

    return raw_scores, fixed_scores, all_labels, all_predictions, all_predictions_fixed

In [15]:
def evaluate(data_loader, model, sents):
    """
    Compute scores given the predictions and gold labels
    """
    device = torch.device(f'cuda:{args.n_gpu}')
    model.model.to(device)

    model.model.eval()
    outputs, targets = [], []
    for batch in tqdm(data_loader):
        # need to push the data to device
        outs = model.model.generate(input_ids=batch['source_ids'].to(device),
                                    attention_mask=batch['source_mask'].to(device),
                                    max_length=args.max_seq_length)

        dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

        outputs.extend(dec)
        targets.extend(target)

    raw_scores, fixed_scores, all_labels, all_preds, all_preds_fixed = compute_scores(outputs, targets, sents)
    results = {'raw_scores': raw_scores, 'fixed_scores': fixed_scores, 'labels': all_labels,
               'preds': all_preds, 'preds_fixed': all_preds_fixed}
    # pickle.dump(results, open(f"{args.output_dir}/results-{args.task}-{args.dataset}-{args.paradigm}.pickle", 'wb'))

#     return raw_scores, fixed_scores
    return results



In [16]:
def return_answer(data_loader, model, sents):
    """
    Compute scores given the predictions and gold labels
    """
    device = torch.device(f'cuda:{args.n_gpu}')
    model.model.to(device)

    model.model.eval()
    outputs, targets = [], []
    for batch in tqdm(data_loader):
        # need to push the data to device
        outs = model.model.generate(input_ids=batch['source_ids'].to(device),
                                    attention_mask=batch['source_mask'].to(device),
                                    max_length=args.max_seq_length)

        dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

        outputs.extend(dec)
        targets.extend(target)

    return outputs, targets
#     raw_scores, fixed_scores, all_labels, all_preds, all_preds_fixed = compute_scores(outputs, targets, sents, paradigm, task)




In [17]:
logger = logging.getLogger(__name__)

In [18]:
def get_dataset(tokenizer, type_data, args):
    return ABSADataset(tokenizer=tokenizer, data_path=args.data_path, data_type=type_data, max_len=args.max_seq_length)

def get_dataset_valid(tokenizer, type_data, args, is_valid):
    return ABSADataset(tokenizer=tokenizer, data_path=args.data_path, data_type=type_data, max_len=args.max_seq_length, is_valid=is_valid)


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()

        for key in hparams.__dict__.keys():
            self.hparams[key]=hparams.__dict__[key]

        self.outputs = []
#         self.model = MT5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.model_name_or_path)

    def is_logger(self):
        return True

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None,
                decoder_attention_mask=None, labels=None):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        self.log('train_loss', loss, prog_bar=True)
        return {"loss": loss, "log": tensorboard_logs}

    def on_train_epoch_end(self):
        avg_train_loss = torch.stack(self.outputs).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}


        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.outputs.append(loss)

        self.log('val_loss', loss, prog_bar=True)

        return {"val_loss": loss}


    def configure_optimizers(self):
        '''Prepare optimizer and schedule (linear warmup and decay)'''
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

        self.opt = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)



        self.opt2 = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.opt,
            mode='min',
            factor=0.95,
            patience=0,
            min_lr=1e-6,
            verbose=True
        )


#         return [self.opt], [self.opt2]
        return (
            {"optimizer": self.opt, "lr_scheduler": {"scheduler": self.opt2, "monitor": "val_loss"}}
        )

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
        optimizer.step(optimizer_idx)
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.4f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}
        return tqdm_dict

    def train_dataloader(self):
        print("load_dataset")
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_data="train", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.batch_size, drop_last=True, shuffle=True, num_workers=self.hparams.num_workers)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.batch_size * max(1, len(self.hparams.n_gpu))))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )

        self.lr_scheduler = scheduler

        return dataloader

    def on_validation_epoch_end(self):

        avg_loss = torch.stack(self.outputs).mean()
        tensorboard_logs = {"val_loss": avg_loss}

#         self.train_dataloader()
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}


    def val_dataloader(self):
        print("load_valid_dataset")
        val_dataset = get_dataset_valid(tokenizer=self.tokenizer, type_data="test", args=self.hparams, is_valid=True)
        return DataLoader(val_dataset, batch_size=self.hparams.batch_size, num_workers=self.hparams.num_workers)



In [20]:
class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):

        print("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
        # Log results
        for key in sorted(metrics):
            if key not in ["log", "progress_bar"]:
#                 logger.info("{} = {}\n".format(key, str(metrics[key])))
                print("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        print("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

        # Log and save results to file
#         output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
#         with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
            if key not in ["log", "progress_bar"]:
                print("{} = {}\n".format(key, str(metrics[key])))
#                     writer.write("{} = {}\n".format(key, str(metrics[key])))

In [21]:
seed_everything(args.seed)

tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [22]:
print("\n****** Conduct Training ******")
model = T5FineTuner(args)


****** Conduct Training ******


In [23]:
import tensorflow as tf

checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=args.output_dir, monitor='val_loss', mode='min', save_top_k=1, verbose=1)
earlyStopping= pl.callbacks.EarlyStopping(
        monitor='val_loss', patience=3, mode='min', min_delta=0.001, verbose=1)


train_params = dict(
        default_root_dir=args.output_dir,
        accumulate_grad_batches=args.gradient_accumulation_steps,
        gradient_clip_val=1.0,
        max_epochs=args.num_train_epochs,
        callbacks=[LoggingCallback(), checkpoint_callback, earlyStopping],
)

In [24]:
trainer = pl.Trainer(**train_params)
trainer.fit(model)
print("Finish training and saving the model!")

Sanity Checking: 0it [00:00, ?it/s]

load_valid_dataset
***** Validation results *****
val_loss = tensor(13.2052, device='cuda:0')

load_dataset


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

***** Validation results *****
train_loss = tensor(10.6100, device='cuda:0')

val_loss = tensor(2.9949, device='cuda:0')



...

***** Validation results *****
train_loss = tensor(0.8777, device='cuda:0')

val_loss = tensor(0.4667, device='cuda:0')

Epoch 00012: reducing learning rate of group 0 to 2.2800e-05.
Epoch 00012: reducing learning rate of group 1 to 2.2800e-05.


Validation: 0it [00:00, ?it/s]

***** Validation results *****
train_loss = tensor(0.7908, device='cuda:0')

val_loss = tensor(0.4655, device='cuda:0')

Epoch 00013: reducing learning rate of group 0 to 8.8667e-06.
Epoch 00013: reducing learning rate of group 1 to 8.8667e-06.


Validation: 0it [00:00, ?it/s]

***** Validation results *****
train_loss = tensor(0.5794, device='cuda:0')

val_loss = tensor(0.4638, device='cuda:0')

Finish training and saving the model!


In [25]:
print("\n****** Conduct Evaluating ******")

    # model = T5FineTuner(args)
dev_results, test_results = {}, {}
best_f1, best_checkpoint, best_epoch = -999999.0, None, None
checkpoints=""

    # retrieve all the saved checkpoints for model selection
saved_model_dir = args.output_dir

for f in os.listdir(saved_model_dir):
    file_name = os.path.join(saved_model_dir, f)
    if 'ckpt' in file_name:
        checkpoints = file_name

print(f"We will perform validation on the following checkpoints: {checkpoints}")


****** Conduct Evaluating ******
We will perform validation on the following checkpoints: ./outputs/epoch=10-step=121.ckpt


In [26]:
tokenizer = AutoTokenizer.from_pretrained('google/mt5-base')
valid_dataset = ABSADataset(tokenizer, data_path=args.data_path, data_type='test',max_len=args.max_seq_length)
valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, num_workers=args.num_workers)

# test_dataset = ABSADataset(tokenizer, data_path=args.data_path, data_type='test', max_len=args.max_seq_length)
# test_loader = DataLoader(test_dataset, batch_size=args.batch_size, num_workers=args.num_workers)


In [27]:
model_ckpt = torch.load(checkpoints)


In [28]:
model.load_state_dict(model_ckpt['state_dict'])

<All keys matched successfully>

In [29]:
sents, _ = read_data_from_path(f'{args.data_path}/test', is_valid=True)


In [30]:
valid_result = evaluate(valid_loader, model, sents)


100%|██████████| 82/82 [01:43<00:00,  1.26s/it]


Results of raw output
{'precision': 0, 'recall': 0, 'f1': 0}

Results of fixed output
{'precision': 0, 'recall': 0, 'f1': 0}


In [31]:
valid_answer, valid_gold = return_answer(valid_loader, model, sents)


100%|██████████| 82/82 [01:41<00:00,  1.24s/it]


In [32]:
valid_answer

['và kích thước màn hình > ngoài ra > kích thước > đáng chú ý;',
 'xiaomi > galaxy > màn hình 6,8 inch > lớn hơn đáng kể;xiaomi > galaxy > galaxy > lớn hơn đáng kể;',
 'galaxy > galaxy > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > c',
 'samsung > cả hai điện thoại > hiệu suất thực tế > không đáng kể;samsung > cả hai điện thoại > unknow > hiệu suất thực tế > không đáng kể;',
 'samsung > unknow > xung nhịp > cao hơn một chút;phiên bản snapdragon của samsung > unknow > xung nhịp > cao hơn một chút;',
 'xiaomi 13 pro > unknow > khả năng thu phóng > tốt hơn;xiaomi 13 pro > unknow > unknow > hoạt động tốt hơn;',
 'galaxy 13 pro > galaxy 13 pro > kích thước pin > lớn hơn gần 4%;galaxy 13 pro > galaxy 13 pro > viên pin 4.820 mah trên xiaomi 13 pro > lớn hơn gần 4%;',
 'xi

In [33]:
valid_gold

['unknow > unknow > kích thước > khác biệt đáng chú ý;unknow > unknow > kích thước màn hình > khác biệt đáng chú ý;',
 'galaxy > xiaomi > màn hình > lớn hơn đáng kể;galaxy > xiaomi > tỷ lệ khung hình > rộng hơn;',
 'cả hai > cả hai > máy quét dấu vân tay dưới màn hình > đều dựa vào;cả hai > cả hai > sinh trắc học > đều dựa vào máy quét dấu vân tay dưới màn hình;',
 'cả hai điện thoại > cả hai điện thoại > bộ vi xử lý snapdragon 8 gen 2 của qualcomm > đều đi kèm;cả hai điện thoại > cả hai điện thoại > hiệu suất thực tế > sự khác biệt về hiệu suất thực tế là không đáng kể;',
 'samsung > unknow > phiên bản snapdragon > xung nhịp cao hơn một chút;phiên bản snapdragon của samsung > unknow > xung nhịp > cao hơn một chút;',
 'xiaomi 13 pro > galaxy > cảm biến máy ảnh chính > lớn hơn;xiaomi 13 pro > galaxy > cảm biến máy ảnh chính > hoạt động tốt hơn vào ban đêm;',
 'xiaomi 13 pro > galaxy > kích thước pin > lớn hơn gần 4%;galaxy > xiaomi 13 pro > pin > dễ dàng giành chiến thắng;',
 'galaxy > 

In [34]:
def compare(val_answer, val_gold):
    count = 0
    count1 = 0
    for i in range(len(val_answer)):
        if(val_answer[i] != val_gold[i]):    
            print(val_answer[i])
            print('---------')
            print(val_gold[i])
            count+=1
        else:
            count1 +=1
        print('=====')
    print(count, count1)

In [35]:
compare(valid_answer, valid_gold)

và kích thước màn hình > ngoài ra > kích thước > đáng chú ý;
---------
unknow > unknow > kích thước > khác biệt đáng chú ý;unknow > unknow > kích thước màn hình > khác biệt đáng chú ý;
=====
xiaomi > galaxy > màn hình 6,8 inch > lớn hơn đáng kể;xiaomi > galaxy > galaxy > lớn hơn đáng kể;
---------
galaxy > xiaomi > màn hình > lớn hơn đáng kể;galaxy > xiaomi > tỷ lệ khung hình > rộng hơn;
=====
galaxy > galaxy > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > cả hai > c
---------
cả hai > cả hai > máy quét dấu vân tay dưới màn hình > đều dựa vào;cả hai > cả hai > sinh trắc học > đều dựa vào máy quét dấu vân tay dưới màn hình;
=====
samsung > cả hai điện thoại > hiệu suất thực tế > không đáng kể;samsung > cả hai điện thoại > unknow > hiệu suất thực tế > không đáng kể;
---------
